In [1]:
import pandas as pd
import numpy as np
from math import sqrt
from numpy import split
from numpy import array
from pandas import read_csv
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.tsa.arima_model import ARIMA

In [6]:
dataset = pd.read_csv('household_power_consumption.txt', sep=';', header=0, low_memory=False,
infer_datetime_format=True, parse_dates={'datetime':[0,1]}, index_col=['datetime'])

In [7]:
dataset.head(15)

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
datetime,,,,,,,
2006-12-16 17:24:00,4.216,0.418,234.840,18.400,0.000,1.000,17.0
2006-12-16 17:25:00,5.360,0.436,233.630,23.000,0.000,1.000,16.0
2006-12-16 17:26:00,5.374,0.498,233.290,23.000,0.000,2.000,17.0
2006-12-16 17:27:00,5.388,0.502,233.740,23.000,0.000,1.000,17.0
2006-12-16 17:28:00,3.666,0.528,235.680,15.800,0.000,1.000,17.0
2006-12-16 17:29:00,3.520,0.522,235.020,15.000,0.000,2.000,17.0
2006-12-16 17:30:00,3.702,0.520,235.090,15.800,0.000,1.000,17.0
2006-12-16 17:31:00,3.700,0.520,235.220,15.800,0.000,1.000,17.0
2006-12-16 17:32:00,3.668,0.510,233.990,15.800,0.000,1.000,17.0


In [13]:
values_array=dataset.values

In [15]:
values_array.shape
# dataset.shape

(2075259, 7)

In [19]:
values_array[0]

array(['4.216', '0.418', '234.840', '18.400', '0.000', '1.000', 17.0],
      dtype=object)

In [22]:
r=values_array[0].reshape(1,1,7)

In [39]:
# samples, time steps, features
values_areshaped=values_array.reshape(691753,3,7)

In [43]:
df2=pd.DataFrame
df2.values=values_areshaped
df2.columns=dataset.columns

In [50]:
df2.values[0]

array([['4.216', '0.418', '234.840', '18.400', '0.000', '1.000', 17.0],
       ['5.360', '0.436', '233.630', '23.000', '0.000', '1.000', 16.0],
       ['5.374', '0.498', '233.290', '23.000', '0.000', '2.000', 17.0]],
      dtype=object)

In [51]:
df2.values[1]

array([['5.388', '0.502', '233.740', '23.000', '0.000', '1.000', 17.0],
       ['3.666', '0.528', '235.680', '15.800', '0.000', '1.000', 17.0],
       ['3.520', '0.522', '235.020', '15.000', '0.000', '2.000', 17.0]],
      dtype=object)

In [53]:
from numpy import array
# define the dataset
data = list()
n = 5000
for i in range(n):
    data.append([i+1, (i+1)*10])
data = array(data)
# drop time
data = data[:, 1]
# split into samples (e.g. 5000/200 = 25)
samples = list()
length = 200
# step over the 5,000 in jumps of 200
for i in range(0,n,length):
    # grab from i to i + 200
    sample = data[i:i+length]
    samples.append(sample)
# convert list of arrays into 2d array
data = array(samples)
# reshape into [samples, timesteps, features]
data = data.reshape((len(samples), length, 1))
print(data.shape)

(25, 200, 1)


In [71]:
# data[1]

In [2]:
from numpy import array
from numpy import hstack
from keras.models import Sequential
from keras.layers import Dense
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the dataset
        if end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)
        # define input sequence


ImportError: Keras requires TensorFlow 2.2 or higher. Install TensorFlow via `pip install tensorflow`

In [87]:
in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = array([in_seq1[i]+  in_seq2[i] for i in range(len(in_seq1))   ])
# convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))
# horizontally stack columns
dataset = hstack((in_seq1, in_seq2, out_seq))
# choose a number of time steps
n_steps = 3
# convert into input/output
X, y = split_sequences(dataset, n_steps)
# flatten input
n_input = X.shape[1] * X.shape[2]
X = X.reshape((X.shape[0], n_input))
# define model
model = Sequential()
model.add(Dense(100, activation='relu', input_dim=n_input))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=2000, verbose=0)
# demonstrate prediction
x_input = array([[80, 85], [90, 95], [100, 105]])
x_input = x_input.reshape((1, n_input))
yhat = model.predict(x_input, verbose=0)
print(yhat)

1/1 [==============================] - 0s 42ms/step
[[205.31685]]


In [84]:
X

array([[10, 15, 20, 25, 30, 35],
       [20, 25, 30, 35, 40, 45],
       [30, 35, 40, 45, 50, 55],
       [40, 45, 50, 55, 60, 65],
       [50, 55, 60, 65, 70, 75],
       [60, 65, 70, 75, 80, 85],
       [70, 75, 80, 85, 90, 95]])

In [86]:
x_input

array([[ 80,  85,  90,  95, 100, 105]])

In [29]:
x= array([[5.1, 3.5, 1.4, 0.2],
          [4.9, 3. , 1.4, 0.2],
          [4.7, 3.2, 1.3, 0.2],
          [4.6, 3.1, 1.5, 0.2],
          [5. , 3.6, 1.4, 0.2],
          [5.4, 3.9, 1.7, 0.4],
          [4.6, 3.4, 1.4, 0.3],
          [5. , 3.4, 1.5, 0.2],
          [4.4, 2.9, 1.4, 0.2]])



In [30]:
# x[:,:] # would mean u want every row and every column.

# x[3,3] #would mean u want the 3 row and the 3 column value

x[:,:,0]# would mean u want the rows and columns until 3

# x[:, 3] #would mean u want the 3 column and every row

IndexError: too many indices for array: array is 2-dimensional, but 3 were indexed

In [7]:
scores =[0.123123,0.123123,0.5244325]
score=1.123
s_scores=', '.join(['%.1f' % s for s in scores])
print('%s: [%.3f] %s' % ('name', score, s_scores))

name: [1.123] 0.1, 0.1, 0.5
